[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pdf-tools/components-code-sample-hub/blob/main/jupyter/pdftools_sdk/pdftools_sdk_validate_simple.ipynb)

In [ ]:
%pip install pdftools_sdk
%pip install ipython

# Validate PDF conformance
Assess whether a PDF document adheres to specific
standards and conformance levels.

In [ ]:
import io
from pdftools_sdk.pdf import Document, Conformance
from pdftools_sdk.pdf_a.validation import Validator, ErrorCategory

In [ ]:
# Download a file from a given URL and save it to the local system
def prepare_file(url: str, path: str):
    import requests
    response = requests.get(url)
    response.raise_for_status()

    with open(path, 'wb') as f:
        f.write(response.content)

In [ ]:
# Set input arguments
input_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/PdfPrimerWhitepaper.pdf'
input_file_path = 'PdfPrimerWhitepaper.pdf'
prepare_file(input_url, input_file_path)

In [ ]:
def error_listener(context, data_part: str, message: str, category: ErrorCategory, context_text: str, page_no: int, object_no: int):
    if page_no > 0:
        print(f"- {category.name}: {message.decode()} ({context_text.decode()} on page {page_no})")
    else:
        print(f"- {category.name}: {message.decode()} ({context_text.decode()})")

In [ ]:
def validate(input_file_path: str):
    # Open the document
    with io.FileIO(input_file_path, 'rb') as in_stream:
        with Document.open(in_stream) as input_document:
            # Create a validator object that writes all validation error messages to the console
            validator = Validator()
            validator.add_error_handler(error_listener)

            # Validate the standard conformance of the document
            return validator.validate(input_document)

In [ ]:
try:
    # By default, a test license key is active. In this case, a watermark is added to the output. 
    # If you have a license key, please uncomment the following call and set the license key.
    # from pdftools_sdk.sdk import Sdk
    # Sdk.initialize("INSERT-LICENSE-KEY")

    validation_result = validate(input_file_path)
    
    # Report the validation result
    if validation_result.is_conforming:
        print(f"Document conforms to {Conformance(validation_result.conformance).name}.")
    else:
        print(f"Document does not conform to {Conformance(validation_result.conformance).name}.")

    print(f"Validation of {input_file_path} finished.")
except Exception as e:
    print(f"An error occurred: {e}")